### Required Libraries

In [1]:
# beautifulsoup4      4.9.3
# fake-useragent      0.1.11
# helium              3.0.5
# ipython             7.22.0
# matplotlib          3.3.4
# pandas              1.2.4
# Pillow              8.2.0
# requests            2.25.1
# tqdm                4.59.0

# !pip install beautifulsoup4
# !pip install fake-useragent
# !pip install helium
# !pip install ipython
# !pip install Pillow
# !pip install requests
# !pip install tqdm
# !pip install selenium
# !pip install webdriver-manager

# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [2]:
from helium import *
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm as tq
import time
import requests
import os
from fake_useragent import UserAgent

prod = "MEASURING_TOOLS"

In [3]:
df_category = pd.read_csv(f'files/{prod}.csv')
df_category.head()

,product_name,product_link_name,link,major_category_name,major_category_link
0,MEASURING_TOOLS,LEVELS,https://www.canadiantire.ca/en/tools-hardware/...,MEASURING TOOLS,https://www.canadiantire.ca/en/tools-hardware/...
1,MEASURING_TOOLS,SQUARES AND RULERS,https://www.canadiantire.ca/en/tools-hardware/...,MEASURING TOOLS,https://www.canadiantire.ca/en/tools-hardware/...
2,MEASURING_TOOLS,STUD FINDERS,https://www.canadiantire.ca/en/tools-hardware/...,MEASURING TOOLS,https://www.canadiantire.ca/en/tools-hardware/...
3,MEASURING_TOOLS,TAPE MEASURES,https://www.canadiantire.ca/en/tools-hardware/...,MEASURING TOOLS,https://www.canadiantire.ca/en/tools-hardware/...
4,MEASURING_TOOLS,SPECIALTY MEASURING TOOLS,https://www.canadiantire.ca/en/tools-hardware/...,MEASURING TOOLS,https://www.canadiantire.ca/en/tools-hardware/...


In [4]:
def update_link(link):
    split = link.split('?') 
    complement = 'https:'
    updated_link = complement + split[0]
    return updated_link

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
ua = UserAgent()

import random
randomlist = []
for i in range(0,5):
    n = random.randint(110, 150)
    randomlist.append(n/10)

df_list = []
for i in tq(range(len(df_category))):
    print(i)
    product_name = df_category['product_name'][i]
    class_name = df_category['product_link_name'][i]
    url = df_category['link'][i]
    
    print('Start fetching process for {}!'.format(class_name))
    
    if url[-3:] != '_no':
        number_of_pages = 5 
        url_list = [url + 'page={}'.format(i) for i in range(1, number_of_pages+1)]
    else:
        url_list = [url[:-3]]
    
    soup_list = []
    count = 0
    for page in tq(url_list):
        start = time.time()
        user_agent = ua.chrome
        options.add_argument('--user-agent="{}"'.format(user_agent))
        browser = start_chrome(page, headless=True, options=options)
        time.sleep(1)
        #webdriver.findElement(By.cssSelector("body")).sendKeys(Keys.ESCAPE);
        press(ESCAPE)
        time.sleep(5)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        #checker = soup.find_all('div', {'class':'srp-grid__item srp-grid__item_pcode'})
        checker = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        if len(checker) > 0:
            soup_list.append(soup)
            print('Page fetched: {}.'.format(page))
        elif (len(checker)==0) & (count < 2):
            count += 1
            print('Page not fetched: {}.'.format(page))
        else:
            print('Counter: ', count)
            print('Page not fetched: {}.'.format(page))
            print("No more pages avaiable")
            kill_browser()
            sleeping = random.choice(randomlist)
            time.sleep(sleeping)  
            end = time.time()
            time_elapsed = round(end - start, 1)
            print('Time elapsed: {}s'.format(time_elapsed))
            print('\n')
            try:
                kill_browser()
                print('Browser closed.')
            except:
                print('There were no browser open.')
                pass
            break

        try:
            kill_browser()
            print('Browser closed.')
        except:
            print('There were no browser open.')
            pass

        sleeping = random.choice(randomlist)
        time.sleep(sleeping)  
        end = time.time()
        time_elapsed = round(end - start, 1)
        print('Time elapsed: {}s'.format(time_elapsed))
    print('Fetching process for {} finished!'.format(class_name))
    print('Creating dataframe.')
    
    image_link_list = []
    original_name_list = []

    for soup in soup_list: 
        #products_list = soup.find_all('div', {'class':'srp-grid__item srp-grid__item_pcode'})
        products_list = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        for product in products_list:
            image = product.find('img')
            link = image['src']
            link = update_link(link)
            original_name = image['alt']
            image_link_list.append(link)
            original_name_list.append(original_name)

    df_temp = pd.DataFrame({'original_name': original_name_list, 'image_link': image_link_list})
    df_temp['class_name'] = class_name
    df_temp['product_name'] = product_name
    df_list.append(df_temp)
    print('Dataframe for {} created.\n\n\n\n'.format(class_name))

print('Creating final dataset.')
df_final = pd.concat(df_list)
df_final.drop_duplicates(subset='image_link', inplace=True)
df_final.reset_index(inplace=True)
del df_final['index']
print('Process finished. Printing final dataset.')
df_final

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0
Start fetching process for LEVELS!
Page fetched: https://www.canadiantire.ca/en/tools-hardware/hand-tools/measuring-tools/levels.html?adlocation=LIT_Category_Product_LevelsCat_en;page=1.
Browser closed.



 20%|████████████████▊                                                                   | 1/5 [00:25<01:42, 25.59s/it]

Time elapsed: 25.6s
Page fetched: https://www.canadiantire.ca/en/tools-hardware/hand-tools/measuring-tools/levels.html?adlocation=LIT_Category_Product_LevelsCat_en;page=2.
Browser closed.



 40%|█████████████████████████████████▌                                                  | 2/5 [00:53<01:20, 26.79s/it]

...


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:37<00:24, 24.69s/it]

Time elapsed: 24.2s
Page fetched: https://www.canadiantire.ca/en/tools-hardware/hand-tools/measuring-tools/flagging-tape.html?adlocation=LIT_Category_Product_FlaggingTapeCat_enpage=5.
Browser closed.



100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [17:03<00:00, 127.92s/it]

Time elapsed: 24.3s
Fetching process for FLAGGING TAPE finished!
Creating dataframe.
Dataframe for FLAGGING TAPE created.




Creating final dataset.
Process finished. Printing final dataset.


,original_name,image_link,class_name,product_name
0,"MAXIMUM Magnetic Box Beam Level, 78-in",https://canadiantire.scene7.com/is/image/Canad...,LEVELS,MEASURING_TOOLS
1,Mastercraft Aluminum Box Level,https://canadiantire.scene7.com/is/image/Canad...,LEVELS,MEASURING_TOOLS
2,"MAXIMUM Magnetic Box Beam Level, 48-in",https://canadiantire.scene7.com/is/image/Canad...,LEVELS,MEASURING_TOOLS
3,Mastercraft 12-in. Magnetic Level,https://canadiantire.scene7.com/is/image/Canad...,LEVELS,MEASURING_TOOLS
4,"Mastercraft I-Beam Level, 48-in",https://canadiantire.scene7.com/is/image/Canad...,LEVELS,MEASURING_TOOLS
...,...,...,...,...
138,Irwin Mach6 Reel Combo with Blue Chalk,https://canadiantire.scene7.com/is/image/Canad...,CHALK AND CHALK LINES,MEASURING_TOOLS
139,"Mossy Oak Flagging Tape, 3-pk",https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
140,Mossy Oak™ Standard Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
141,Stanley Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS


In [6]:
df_final.to_csv(f'files/{prod}_final.csv', index=False)
df_final.tail() 

,original_name,image_link,class_name,product_name
138,Irwin Mach6 Reel Combo with Blue Chalk,https://canadiantire.scene7.com/is/image/Canad...,CHALK AND CHALK LINES,MEASURING_TOOLS
139,"Mossy Oak Flagging Tape, 3-pk",https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
140,Mossy Oak™ Standard Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
141,Stanley Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
142,Mossy Oak™ Reflective Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS


## Saving Images

In [7]:
file = f'files/{prod}_final.csv'
df_final = pd.read_csv(file)
df_final.tail()

,original_name,image_link,class_name,product_name
138,Irwin Mach6 Reel Combo with Blue Chalk,https://canadiantire.scene7.com/is/image/Canad...,CHALK AND CHALK LINES,MEASURING_TOOLS
139,"Mossy Oak Flagging Tape, 3-pk",https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
140,Mossy Oak™ Standard Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
141,Stanley Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS
142,Mossy Oak™ Reflective Flagging Tape,https://canadiantire.scene7.com/is/image/Canad...,FLAGGING TAPE,MEASURING_TOOLS


In [8]:
def get_full_link(link, image_number):    
    complement = '?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700'
    link = link[:-1]
    full_link = "{}{}{}".format(link, image_number, complement)
    return full_link

In [ ]:
import os
total_images = 0
start = time.time()
links_not_fetched = []
for i in range(len(df_final)):
    product_name = df_final['class_name'][i]
    new_name = "{}_{}".format(product_name, i)
    original_name = df_final['original_name'][i]
    directory = f'images/{prod}'
    link = df_final['image_link'][i]
    
    try:
        os.makedirs('{}/{}'.format(directory, product_name))
    except OSError:
        pass
    
    print("\n\n Fetching images of: ", product_name, '\n')
    count = 0
    for i in range(1, 11):
        full_link = get_full_link(link, i)
        print('Link: ', full_link)
        file_name = '{}/{}/{}_photo_{}.jpg'.format(directory, product_name, new_name, i)
        img = requests.get(full_link)      
    
        if len(img.content) < 7000:
            print('No more product images.')
            print('Number of images saved: ', count)
            break
        else:
            count += 1
            total_images += 1
            print('Saving product image.')
            with open(file_name, 'wb') as f:
                f.write(img.content)
                time.sleep(0.5)
    if count == 0:
        links_not_fetched.append(full_link)
        print('\n\n', '########### \n', 'No image in the link: ', full_link, '\n #########','\n\n')
        
end = time.time()
time_elapsed = round(end - start, 1)
print('Time elapsed: {}s'.format(time_elapsed))
print('\n')
print('Total number of images: ', total_images)     



 Fetching images of:  LEVELS 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0575621_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  0


 ########### 
 No image in the link:  https://canadiantire.scene7.com/is/image/CanadianTire/0575621_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700 
 ######### 




 Fetching images of:  LEVELS 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0575330_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0575330_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0575330_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  2


 Fetching images of:  LEVELS 

Link:  https://canadiantire

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574661_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574661_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  2


 Fetching images of:  LEVELS 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574651_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574651_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  LEVELS 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0575369_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/057

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577271_6?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  5


 Fetching images of:  SQUARES AND RULERS 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577834_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577834_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  SQUARES AND RULERS 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577827_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577827_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/i

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574645_5?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574645_6?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  5


 Fetching images of:  STUD FINDERS 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574656_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574656_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574656_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0574656_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577117_6?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577117_7?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577117_8?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577117_9?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  8


 Fetching images of:  TAPE MEASURES 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577243_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577243_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Savin

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577114_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577114_5?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577114_6?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577114_7?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577114_8?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0577114_9?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  8


 Fetching images of: 